In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

2022-06-14 10:07:51.024620: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-14 10:07:51.024746: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [17]:
import pandas as pd

In [18]:
df = pd.read_csv('spam.csv',encoding = "latin-1")

In [19]:
df

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [20]:
df.drop(["Unnamed: 2", 'Unnamed: 3', 'Unnamed: 4'], inplace=True, axis = 1)

In [26]:
df.rename(columns={'v1':'Category', 'v2':'Message'} )

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [27]:
df.sample(5)

,Category,Message
1305,ham,Designation is software developer and may be s...
2303,ham,Should I tell my friend not to come round til ...
1965,ham,Thanks. It was only from tescos but quite nice...
560,ham,Hi msg me:)i'm in office..
4818,ham,Got smaller capacity one? Quite ex...


In [28]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    653  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [29]:
#Imbalance Dataset  
747/4825

0.15481865284974095

In [30]:
#15% spam mail and 85% ham mail class imbalance

In [33]:
df.Category = df.Category.apply(lambda x: 1 if x=='spam' else 0)

In [34]:
df

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


In [35]:
df_spam = df[df.Category==1]

In [37]:
df_spam.shape

(747, 2)

In [38]:
df_ham = df[df.Category==0]

In [40]:
df_ham.shape

(4825, 2)

In [41]:
df_ham_balance = df_ham.sample(df_spam.shape[0])
df_ham_balance.shape

(747, 2)

In [42]:
df_ham_balance

,Category,Message
681,0,ÌÏ log off 4 wat. It's sdryb8i
4017,0,"\GRAN ONLYFOUND OUT AFEW DAYS AGO.CUSOON HONI\"""""
298,0,I cant pick the phone right now. Pls send a me...
1177,0,Just nw i came to hme da..
1544,0,After completed degree. There is no use in joi...
...,...,...
3905,0,Went fast asleep dear.take care.
3351,0,Call him and say you not coming today ok and t...
2362,0,How are you doing? Hope you've settled in for ...
376,0,You could have seen me..i did't recognise you ...


In [44]:
df_balance = pd.concat([df_spam,df_ham_balance])
df_balance

,Category,Message
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
5,1,FreeMsg Hey there darling it's been 3 week's n...
8,1,WINNER!! As a valued network customer you have...
9,1,Had your mobile 11 months or more? U R entitle...
11,1,"SIX chances to win CASH! From 100 to 20,000 po..."
...,...,...
3905,0,Went fast asleep dear.take care.
3351,0,Call him and say you not coming today ok and t...
2362,0,How are you doing? Hope you've settled in for ...
376,0,You could have seen me..i did't recognise you ...


In [45]:
df1 = df_balance.copy()
df1.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
0            747    726                             Sorry, I'll call later   
1            747    653  Please call our customer service representativ...   

               
         freq  
Category       
0           6  
1           4

In [47]:
#split into train test data
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = train_test_split(df1.Message, df1.Category, test_size = 0.2, random_state=42)
X_train.shape

(1195,)

In [50]:
#Let's use the bert model to vectorise the text
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

2022-06-14 10:33:32.501796: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-14 10:33:32.501946: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-14 10:33:32.502056: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sourav-HP-Laptop-15-bs0xx): /proc/driver/nvidia/version does not exist
2022-06-14 10:33:32.502999: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [54]:
def text_embedding(text):
    preprocess_text = bert_preprocess(text)
    return bert_encoder(preprocess_text)['pooled_output']

In [55]:
print(text_embedding(["500$ discount. hurry up", 
    "Bhavin, are you up for a volleybal game tomorrow?"]))

tf.Tensor(
[[-0.8435169  -0.51327264 -0.88845724 ... -0.74748874 -0.75314736
   0.91964483]
 [-0.8720836  -0.5054398  -0.9444668  ... -0.85847515 -0.7174535
   0.88082975]], shape=(2, 768), dtype=float32)


In [77]:
test_txt = ['I love the movie', "I love python programming"]
preprocess_text = bert_preprocess(test_txt)
preprocess_text.keys()

dict_keys(['input_word_ids', 'input_type_ids', 'input_mask'])

In [78]:
preprocess_text['input_mask']

#CLS  token 101
#SEP token  102

<tf.Tensor: shape=(2, 128), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int32)>

In [79]:
encoded_text = bert_encoder(preprocess_text)

In [80]:
encoded_text.keys()

dict_keys(['encoder_outputs', 'default', 'pooled_output', 'sequence_output'])

In [81]:
encoded_text['encoder_outputs'][-1] == encoded_text['sequence_output']

<tf.Tensor: shape=(2, 128, 768), dtype=bool, numpy=
array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]]])>

Get embedding vector for some simple words and 

In [88]:
e =text_embedding([
    "banana", 
    "grapes",
    "mango",
    "jeff bezos",
    "elon musk",
    "bill gates"
]
)

In [99]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[1]],[e[5]])

array([[0.96105796]], dtype=float32)

neural Network Functional Model

In [103]:
#Bert Model
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocess_text)
#neural network
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

ValueError: Output tensors of a Functional model must be the output of a TensorFlow `Layer` (thus holding past layer metadata). Found: [[0.7110623 ]
 [0.85928315]]